# Datensammlung von Nachrichtenseiten per Web Scraping - Watson

In [21]:
#Installation Bibliotheken

import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import json
import re

In [22]:
#Definition der Parameter für das Web Scraping 

#Suchwörter in Google Search API
search_queries = ["Künstliche Intelligenz", "AI", "Artificial Intelligence", "KI"]

#Lege ein Startdatum für die automatische Datenerfassung fest
last_day_str = "2024-03-13"

#Zeitintervall von x Tagen vor dem Zieldatum
request_days = 10

#Limit für API-Anfragen
api_request_limit = 10

#Pausenzeit des Data Scraping über Beautiful Soup in Sekunden 
scrap_pause = 3

#Bereinigung der Datumsangabe
last_day = datetime.strptime(last_day_str, "%Y-%m-%d")

#Berechnung des letzten Tages
first_day = last_day - timedelta(days=request_days*api_request_limit) - ((timedelta(days=1)*api_request_limit)-timedelta(days=1))

#Ausgabe des Zeitraums
print(f"Time period: {first_day.strftime('%Y-%m-%d')} until {last_day_str} with {api_request_limit} requests")

Time period: 2023-11-25 until 2024-03-13 with 10 requests


In [23]:
#API-Anfrage

#API Schlüssel
api_key = "???"

#DataFrame für die Ergebnisse erstellen
df_data_all_queries = []

#Berechnung Zeitintervall für jede Schleife
loop_interval = timedelta(days=request_days)

#Schleife bis zur Erreichung der Grenze an API-Anfragen
for i in range(api_request_limit):
    #Berechnen des Start- und Enddatums für die aktuelle Schleife
    end_date = last_day - (loop_interval * i) - (timedelta(days=1) * i) if i > 0 else last_day - (loop_interval * i)
    start_date = end_date - timedelta(days=request_days)

    #Bereinigung von Datumsangaben
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    #Schleife über die Suchbegriffe
    for search_query in search_queries:

        #API-Endpunkt und Parameter
        api_endpoint = "https://www.googleapis.com/customsearch/v1"
        api_params = {
            "key": api_key,
            "cx": "???",
            "q": f"{search_query} before:{end_date_str}",
            "sort": "date",
        }

        #API-Aufruf
        response = requests.get(api_endpoint, params=api_params)
        data = response.json()

        #Daten im DataFrame speichern
        if "items" in data:
            for item in data["items"]:
                link = item["link"]
                title = item["title"]

                #Extrahieren von HTML-Inhalten aus der Website
                article_response = requests.get(link)

                #Pause von x Sekunden
                time.sleep(scrap_pause)

                soup = BeautifulSoup(article_response.text, 'html.parser')

                '''Individueller Teil je Quelle'''

                #Extrahiere den Inhalt
                #Suchen Sie das <div>-Element mit der Klasse "watson-snippet__title"
                snippet_title_div = soup.find('div', class_='watson-snippet__title')

                #Entferne den Text innerhalb des gefundenen <div>-Elements
                if snippet_title_div:
                    snippet_title_div.decompose()

                #Finde alle <p> Tags mit der Klasse "watson-snippet__text"
                paragraphs = soup.find_all('p', class_='watson-snippet__text')

                #Text aus <p>-Tags extrahieren und zu einer Liste hinzufügen
                text_list = [paragraph.get_text(strip=True) for paragraph in paragraphs]

                #Verkette die Texte zu einem zusammenhängenden Artikeltext
                content = '\n'.join(text_list)

                #Extrahieren des Datums aus dem HTML-Inhalt
                script_tag = soup.find('script', {'id': 'datalayer'})

                #Extrahieren des Datums aus dem Skriptinhalt
                data = re.search(r'"publishedDate":"(\d{4}-\d{2}-\d{2})T', script_tag.text)
                formatted_date = data.group(1) if data else "1900-01-01"

                #Titel aus dem HTML-Code extrahieren
                article_title = soup.title.text if soup.title else ""

                '''Individueller Teil je Quelle'''

                #Prüfen, ob das Datum mit dem Enddatum übereinstimmt
                if datetime.strptime(start_date_str, "%Y-%m-%d") <= datetime.strptime(formatted_date, "%Y-%m-%d") <= datetime.strptime(end_date_str, "%Y-%m-%d"):
                    df_data_all_queries.append({
                        "Search Query": search_query,
                        "Quelle": "Watson",
                        "Datum": formatted_date,
                        "Link": link,
                        "Titel": article_title,
                        "Text": content
                    })

#DataFrame für alle Suchbegriffe erstellen
df_all_queries = pd.DataFrame(df_data_all_queries)

#DataFrame anzeigen mit Prüfen, ob der DataFrame leer ist
if df_all_queries.empty:
    print("No entries found!")
else:
    print(df_all_queries)

              Search Query  Quelle       Datum  \
0                       KI  Watson  2024-03-12   
1                       KI  Watson  2024-03-11   
2                       KI  Watson  2024-03-05   
3                       KI  Watson  2024-03-05   
4                       KI  Watson  2024-03-04   
5                       AI  Watson  2024-03-01   
6   Künstliche Intelligenz  Watson  2024-02-19   
7   Künstliche Intelligenz  Watson  2024-02-12   
8                       KI  Watson  2024-02-19   
9                       KI  Watson  2024-02-16   
10                      KI  Watson  2024-02-14   
11                      KI  Watson  2024-02-13   
12                      KI  Watson  2024-02-12   
13                      KI  Watson  2024-02-10   
14  Künstliche Intelligenz  Watson  2024-02-05   
15  Künstliche Intelligenz  Watson  2024-01-31   
16                      AI  Watson  2024-02-02   
17                      AI  Watson  2024-02-02   
18                      KI  Watson  2024-02-09   


In [24]:
#Neue Daten in einem Hauptdatensatz in CSV speichern

#Prüfen, ob die CSV-Datei bereits existiert
try:
    #Versuche, die vorhandene CSV-Datei zu lesen
    existing_df = pd.read_csv("2_Daten_Watson_V2.csv")
except FileNotFoundError:
    #Wenn die Datei nicht existiert, erstelle einen leeren DataFrame
    existing_df = pd.DataFrame()

#Anhängen der neuen Daten an den bestehenden DataFrame
df = pd.DataFrame(df_all_queries)
df_to_append = pd.concat([existing_df, df], ignore_index=True)

#Duplikate auf Basis aller Spalten entfernen
df_to_append = df_to_append.drop_duplicates()

#Speichern des kombinierten DataFrame in der CSV-Datei
df_to_append.to_csv("2_Daten_Watson_V2.csv", index=False)